**Table of contents**<a id='toc0_'></a>    
- [ENSG](#toc1_)    
    - [How many total unique gene records are there in Ensembl](#toc1_1_1_)    
    - [Identify alias-alias collision symbols](#toc1_1_2_)    
- [HGNC](#toc2_)    
    - [How many total unique gene records are there in HGNC](#toc2_1_1_)    
    - [Identify alias-alias collision symbols](#toc2_1_2_)    
- [NCBI Info](#toc3_)    
    - [How many total unique gene records are there in NCBI Gene](#toc3_1_1_)    
    - [Identify alias-alias collision symbols](#toc3_1_2_)    
- [Merge to create Alias-Alias Collision Table- On Primary Gene Symbol](#toc4_)    
- [Merge to create Alias-Alias Collision Table- On Alias Symbol](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [30]:
import pandas as pd
import numpy as np
import plotly.express as px

In [31]:
def create_aa_collision_df(subset_genes_xxxx_df: pd.DataFrame, merged_alias_xxxx_df: pd.DataFrame, source: str) -> pd.DataFrame:
    """Create a df of alias-alias collision symbols 

    :param subset_genes_xxxx_df: Processed df of gene records
    :param source: Representation of the source of the gene records
    :return: A df of genes that share an alias with another gene
    """
    #Find alias_symbols that are shared by multiple different gene_symbols
    dup_alias = (
        subset_genes_xxxx_df.groupby('alias_symbol')['gene_symbol']
        .nunique()
        .reset_index()
    )
    #Keep only alias_symbols linked to more than one gene
    dup_alias = dup_alias[dup_alias['gene_symbol'] > 1]['alias_symbol']

    #Filter the original df for those alias collisions
    aa_collision_xxxx_df = subset_genes_xxxx_df[subset_genes_xxxx_df['alias_symbol'].isin(dup_alias)].copy()
    aa_collision_xxxx_df = aa_collision_xxxx_df.rename(
    columns={"alias_symbol": "collision"})

    aa_collision_xxxx_df = aa_collision_xxxx_df.sort_values("collision")

    #Add a source tag for future merging efforts
    aa_collision_xxxx_df["source"] = str(source)
    aa_collision_xxxx_df.to_csv(f'../output/single_alias_aa_collision_{source.lower()}_df.csv', index=True)

    #Create a secondary collision df that merges the alias symbols for each record
    columns_map = {
    'ENSG': ['NCBI_ID', 'HGNC_ID'],
    'HGNC': ['NCBI_ID', 'ENSG_ID'],
    'NCBI': ['HGNC_ID', 'ENSG_ID']
    }
    cols_of_interest = columns_map.get(source, [])

    merged_alias_aa_collision_xxxx_df = aa_collision_xxxx_df.drop(columns=cols_of_interest)

    merged_alias_aa_collision_xxxx_df = pd.merge(
        merged_alias_aa_collision_xxxx_df,
        merged_alias_xxxx_df[[f"{source}_ID"] + cols_of_interest + ['alias_symbol']],
        on=[f"{source}_ID"],
        how="left"
    )
    #Convert the df into a csv
    merged_alias_aa_collision_xxxx_df.to_csv(f'../output/merged_alias_aa_collision_{source.lower()}_df.csv', index=True)

    return merged_alias_aa_collision_xxxx_df.head()

# <a id='toc1_'></a>[ENSG](#toc0_)

In [32]:
mini_ensg_df = pd.read_csv(
    "../output/mini_ensg_df.csv", index_col=0
)

In [33]:
subset_genes_ensg_df = pd.read_csv(
    "../output/subset_genes_ensg_df.csv", index_col=[0])

In [34]:
merged_alias_ensg_df = pd.read_csv(
    "../output/merged_alias_ensg_df.csv", index_col=[0])

### <a id='toc1_1_1_'></a>[How many total unique gene records are there in Ensembl](#toc0_)

In [35]:
gene_record_set_ensg = set(mini_ensg_df['ENSG_ID'])
gene_record_count_ensg = len(gene_record_set_ensg)
gene_record_count_ensg

48401

### <a id='toc1_1_2_'></a>[Identify alias-alias collision symbols](#toc0_)

In [36]:
create_aa_collision_df(subset_genes_ensg_df, merged_alias_ensg_df, source="ENSG")

,ENSG_ID,collision,gene_symbol,source,NCBI_ID,HGNC_ID,alias_symbol
0,ENSG00000139187,2F1,KLRG1,ENSG,GENE ID:10219,HGNC:6380,"2F1,CLEC15A,MAFA,MAFA-L"
1,ENSG00000005022,2F1,SLC25A5,ENSG,GENE ID:292,HGNC:10991,"2F1,ANT2,T2,T3"
2,ENSG00000163220,60B8AG,S100A9,ENSG,GENE ID:6280,HGNC:10499,"60B8AG,CAGB,CFAG,CGLB,LIAG,MAC387,MIF,MRP-14,M..."
3,ENSG00000143546,60B8AG,S100A8,ENSG,GENE ID:6279,HGNC:10498,"60B8AG,CAGA,CFAG,CGLA,MRP-8,MRP8,P8,S100-A8"
4,ENSG00000065135,87U6,GNAI3,ENSG,GENE ID:2773,HGNC:4387,87U6


In [37]:
merged_alias_aa_collision_ensg_df = pd.read_csv(
    "../output/merged_alias_aa_collision_ensg_df.csv", index_col=[0])
merged_alias_aa_collision_ensg_df

,ENSG_ID,collision,gene_symbol,source,NCBI_ID,HGNC_ID,alias_symbol
0,ENSG00000139187,2F1,KLRG1,ENSG,GENE ID:10219,HGNC:6380,"2F1,CLEC15A,MAFA,MAFA-L"
1,ENSG00000005022,2F1,SLC25A5,ENSG,GENE ID:292,HGNC:10991,"2F1,ANT2,T2,T3"
2,ENSG00000163220,60B8AG,S100A9,ENSG,GENE ID:6280,HGNC:10499,"60B8AG,CAGB,CFAG,CGLB,LIAG,MAC387,MIF,MRP-14,M..."
3,ENSG00000143546,60B8AG,S100A8,ENSG,GENE ID:6279,HGNC:10498,"60B8AG,CAGA,CFAG,CGLA,MRP-8,MRP8,P8,S100-A8"
4,ENSG00000065135,87U6,GNAI3,ENSG,GENE ID:2773,HGNC:4387,87U6
...,...,...,...,...,...,...,...
4721,ENSG00000164756,ZNT8,SLC30A8,ENSG,GENE ID:169026,HGNC:20303,"ZNT-8,ZNT8"
4722,ENSG00000196660,ZRC1,SLC30A10,ENSG,GENE ID:55532,HGNC:25355,"DKFZP547M236,ZNT-10,ZNT10,ZNT8,ZRC1"
4723,ENSG00000170385,ZRC1,SLC30A1,ENSG,GENE ID:7779,HGNC:11012,"ZNT1,ZRC1"
4724,ENSG00000162378,ZYG11,ZYG11B,ENSG,GENE ID:79699,HGNC:25820,"FLJ13456,ZYG11"


How many ambiguous symbols result from alias-alias collisions?

In [38]:
aa_collision_ambiguous_symbol_set_ensg = set(merged_alias_aa_collision_ensg_df["collision"])
aa_collision_ambiguous_symbol_count_ensg = len(aa_collision_ambiguous_symbol_set_ensg)
aa_collision_ambiguous_symbol_count_ensg

1614

How many records have at least one alias-alias collision (alias that matches another record's alias gene symbol)?

In [39]:
aa_record_set_ensg = set(merged_alias_aa_collision_ensg_df["ENSG_ID"])
aa_record_count_ensg = len(aa_record_set_ensg)
aa_record_count_ensg

3778

# <a id='toc2_'></a>[HGNC](#toc0_)

In [40]:
mini_hgnc_df = pd.read_csv(
    "../output/mini_hgnc_df.csv"
)

In [41]:
subset_genes_hgnc_df = pd.read_csv(
    "../output/subset_genes_hgnc_df.csv", index_col=[0])

In [42]:
merged_alias_hgnc_df = pd.read_csv(
    "../output/merged_alias_hgnc_df.csv", index_col=[0])

### <a id='toc2_1_2_'></a>[Identify alias-alias collision symbols](#toc0_)

In [43]:
create_aa_collision_df(subset_genes_hgnc_df, merged_alias_hgnc_df, source="HGNC")

,HGNC_ID,gene_symbol,collision,source,NCBI_ID,ENSG_ID,alias_symbol
0,HGNC:6380,KLRG1,2F1,HGNC,GENE ID:10219,ENSG00000139187,"2F1,CLEC15A,MAFA,MAFA-L"
1,HGNC:10991,SLC25A5,2F1,HGNC,GENE ID:292,ENSG00000005022,"2F1,T2,T3"
2,HGNC:10498,S100A8,60B8AG,HGNC,GENE ID:6279,ENSG00000143546,"60B8AG,CGLA,MRP-8,MRP8,P8,S100-A8"
3,HGNC:10499,S100A9,60B8AG,HGNC,GENE ID:6280,ENSG00000163220,"60B8AG,CGLB,LIAG,MAC387,MIF,MRP-14,MRP14,NIF,P..."
4,HGNC:10230,RNU6V,87U6,HGNC,GENE ID:6071,ENSG00000206832,"87U6,LH87"


In [44]:
merged_alias_aa_collision_hgnc_df = pd.read_csv(
    "../output/merged_alias_aa_collision_hgnc_df.csv", index_col=[0])
merged_alias_aa_collision_hgnc_df

,HGNC_ID,gene_symbol,collision,source,NCBI_ID,ENSG_ID,alias_symbol
0,HGNC:6380,KLRG1,2F1,HGNC,GENE ID:10219,ENSG00000139187,"2F1,CLEC15A,MAFA,MAFA-L"
1,HGNC:10991,SLC25A5,2F1,HGNC,GENE ID:292,ENSG00000005022,"2F1,T2,T3"
2,HGNC:10498,S100A8,60B8AG,HGNC,GENE ID:6279,ENSG00000143546,"60B8AG,CGLA,MRP-8,MRP8,P8,S100-A8"
3,HGNC:10499,S100A9,60B8AG,HGNC,GENE ID:6280,ENSG00000163220,"60B8AG,CGLB,LIAG,MAC387,MIF,MRP-14,MRP14,NIF,P..."
4,HGNC:10230,RNU6V,87U6,HGNC,GENE ID:6071,ENSG00000206832,"87U6,LH87"
...,...,...,...,...,...,...,...
2443,HGNC:33357,TEX28P2,pTEX,HGNC,GENE ID:653363,ENSG00000277008,"CXorf2B,pTEX"
2444,HGNC:33146,PPP4R3C,smk1,HGNC,GENE ID:139420,ENSG00000224960,"FLFL3P,FLJ32867,smk1"
2445,HGNC:20219,PPP4R3A,smk1,HGNC,GENE ID:55671,ENSG00000100796,"FLFL1,FLJ20707,MSTP033,PP4R3,smk-1,smk1"
2446,HGNC:28393,SPATA2L,tamo,HGNC,GENE ID:124044,ENSG00000158792,"MGC26885,tamo"


How many ambiguous symbols result from alias-alias collisions?

In [45]:
aa_collision_ambiguous_symbol_set_hgnc = set(merged_alias_aa_collision_hgnc_df["collision"])
aa_collision_ambiguous_symbol_count_hgnc = len(aa_collision_ambiguous_symbol_set_hgnc)
aa_collision_ambiguous_symbol_count_hgnc

1083

How many records have at least one alias-alias collision (alias that matches another record's alias gene symbol)?

In [46]:
aa_record_set_hgnc = set(merged_alias_aa_collision_hgnc_df["HGNC_ID"])
aa_record_count_hgnc = len(aa_record_set_hgnc)
aa_record_count_hgnc

2175

# <a id='toc3_'></a>[NCBI Info](#toc0_)

In [47]:
mini_ncbi_df = pd.read_csv(
    "../output/mini_ncbi_df.csv", index_col=[0]
)

In [49]:
subset_genes_ncbi_df = pd.read_csv(
    "../output/subset_genes_ncbi_df.csv", index_col=[0])

In [50]:
merged_alias_ncbi_df = pd.read_csv(
    "../output/merged_alias_ncbi_df.csv", index_col=[0])

### <a id='toc3_1_2_'></a>[Identify alias-alias collision symbols](#toc0_)

In [51]:
create_aa_collision_df(subset_genes_ncbi_df, merged_alias_ncbi_df, source="NCBI")

,NCBI_ID,gene_symbol,collision,source,HGNC_ID,ENSG_ID,alias_symbol
0,GENE ID:5728,PTEN,10q23del,NCBI,HGNC:9588,ENSG00000171862,"10q23del,BZS,CWS1,DEC,GLM2,MHAM,MMAC1,PTEN1,PT..."
1,GENE ID:657,BMPR1A,10q23del,NCBI,HGNC:1076,ENSG00000107779,"10q23del,ACVRLK3,ALK-3,ALK3,BMPR-1A,CD292,SKR5"
2,GENE ID:239,ALOX12,12-LOX,NCBI,HGNC:429,ENSG00000108839,"12-LOX,12S-LOX,LOG12"
3,GENE ID:246,ALOX15,12-LOX,NCBI,HGNC:433,ENSG00000161905,"12-LOX,15-LOX,15-LOX-1,LOG15"
4,GENE ID:10219,KLRG1,2F1,NCBI,HGNC:6380,ENSG00000139187,"2F1,CLEC15A,MAFA,MAFA-2F1,MAFA-L,MAFA-LIKE"


In [52]:
merged_alias_aa_collision_ncbi_df = pd.read_csv(
    "../output/merged_alias_aa_collision_ncbi_df.csv", index_col=[0])
merged_alias_aa_collision_ncbi_df

,NCBI_ID,gene_symbol,collision,source,HGNC_ID,ENSG_ID,alias_symbol
0,GENE ID:5728,PTEN,10q23del,NCBI,HGNC:9588,ENSG00000171862,"10q23del,BZS,CWS1,DEC,GLM2,MHAM,MMAC1,PTEN1,PT..."
1,GENE ID:657,BMPR1A,10q23del,NCBI,HGNC:1076,ENSG00000107779,"10q23del,ACVRLK3,ALK-3,ALK3,BMPR-1A,CD292,SKR5"
2,GENE ID:239,ALOX12,12-LOX,NCBI,HGNC:429,ENSG00000108839,"12-LOX,12S-LOX,LOG12"
3,GENE ID:246,ALOX15,12-LOX,NCBI,HGNC:433,ENSG00000161905,"12-LOX,15-LOX,15-LOX-1,LOG15"
4,GENE ID:10219,KLRG1,2F1,NCBI,HGNC:6380,ENSG00000139187,"2F1,CLEC15A,MAFA,MAFA-2F1,MAFA-L,MAFA-LIKE"
...,...,...,...,...,...,...,...
8180,GENE ID:55671,PPP4R3A,smk1,NCBI,HGNC:20219,ENSG00000100796,"FLFL1,KIAA2010,MSTP033,PP4R3,PP4R3A,SMEK1,smk-..."
8181,GENE ID:57223,PPP4R3B,smk1,NCBI,HGNC:29267,ENSG00000275052,"FLFL2,PP4R3B,PSY2,SMEK2,smk1"
8182,GENE ID:139420,PPP4R3C,smk1,NCBI,HGNC:33146,ENSG00000224960,"FLFL3P,PPP4R3CP,SMEK3P,smk1"
8183,GENE ID:9825,SPATA2,tamo,NCBI,HGNC:14681,ENSG00000158480,"PD1,PPP1R145,tamo"


How many ambiguous symbols result from alias-alias collisions?

In [53]:
aa_collision_ambiguous_symbol_set_ncbi = set(merged_alias_aa_collision_ncbi_df["collision"])
aa_collision_ambiguous_symbol_count_ncbi = len(aa_collision_ambiguous_symbol_set_ncbi)
aa_collision_ambiguous_symbol_count_ncbi

3414

How many records have at least one alias-alias collision (alias that matches another record's alias gene symbol)?

In [54]:
aa_record_set_ncbi = set(merged_alias_aa_collision_ncbi_df["NCBI_ID"])
aa_record_count_ncbi = len(aa_record_set_ncbi)
aa_record_count_ncbi

5622

# <a id='toc4_'></a>[Merge to create Alias-Alias Collision Table- On Primary Gene Symbol](#toc0_)

In [55]:
merged_aa_collision_gene_df = pd.concat(
    [
        merged_alias_aa_collision_hgnc_df[["gene_symbol", "alias_symbol", "ENSG_ID", "collision", "source"]],
        merged_alias_aa_collision_ncbi_df[["gene_symbol", "alias_symbol", "ENSG_ID", "collision", "source"]],
        merged_alias_aa_collision_ensg_df[["gene_symbol", "alias_symbol", "ENSG_ID", "collision", "source"]],
    ]
)
merged_aa_collision_gene_df

,gene_symbol,alias_symbol,ENSG_ID,collision,source
0,KLRG1,"2F1,CLEC15A,MAFA,MAFA-L",ENSG00000139187,2F1,HGNC
1,SLC25A5,"2F1,T2,T3",ENSG00000005022,2F1,HGNC
2,S100A8,"60B8AG,CGLA,MRP-8,MRP8,P8,S100-A8",ENSG00000143546,60B8AG,HGNC
3,S100A9,"60B8AG,CGLB,LIAG,MAC387,MIF,MRP-14,MRP14,NIF,P...",ENSG00000163220,60B8AG,HGNC
4,RNU6V,"87U6,LH87",ENSG00000206832,87U6,HGNC
...,...,...,...,...,...
4721,SLC30A8,"ZNT-8,ZNT8",ENSG00000164756,ZNT8,ENSG
4722,SLC30A10,"DKFZP547M236,ZNT-10,ZNT10,ZNT8,ZRC1",ENSG00000196660,ZRC1,ENSG
4723,SLC30A1,"ZNT1,ZRC1",ENSG00000170385,ZRC1,ENSG
4724,ZYG11B,"FLJ13456,ZYG11",ENSG00000162378,ZYG11,ENSG


In [56]:
merged_aa_collision_gene_df.to_csv(
    "../output/merged_aa_collision_gene_df.csv", index=False
)

In [57]:
merged_aa_collision_gene_df.loc[merged_aa_collision_gene_df.collision == "ALP"]

,gene_symbol,alias_symbol,ENSG_ID,collision,source
75,CCL27,"ALP,CTACK,CTAK,ESkine,ILC,PESKY,skinkine",ENSG00000213927,ALP,HGNC
76,PDLIM3,ALP,ENSG00000154553,ALP,HGNC
77,ASRGL1,"ALP,ALP1,FLJ22316",ENSG00000162174,ALP,HGNC
78,SLPI,"ALK1,ALP,BLPI,HUSI,HUSI-I,WAP4,WFDC4",ENSG00000124107,ALP,HGNC
79,ATRNL1,"ALP,FLJ45344,KIAA0534",ENSG00000107518,ALP,HGNC
248,ALPP,"ALP,ALPI,IAP,PALP,PLAP,PLAP-1",ENSG00000163283,ALP,NCBI
249,SLPI,"ALK1,ALP,BLPI,HUSI,HUSI-I,MPI,WAP4,WFDC4",ENSG00000124107,ALP,NCBI
250,CCL27,"ALP,CTACK,CTAK,ESKINE,ILC,PESKY,SCYA27",ENSG00000213927,ALP,NCBI
251,PDLIM3,ALP,ENSG00000154553,ALP,NCBI
252,ASRGL1,"ALP,ALP1,CRASH",ENSG00000162174,ALP,NCBI
